In [ ]:
import os
import numpy as np
import pandas as pd



import pickle # saving and loading trained model
from os import path

# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# importing library for plotting
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

import tensorflow as tf
from tensorflow import keras

import tensorflow  as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [ ]:
# Load the data
data_clean = pd.read_csv('cleansample_cicids2017.csv')# use each data set one by one

# # Load the data
# data_clean = pd.read_csv('cleansample_ciciot23.csv')

# # Load the data
# data_clean = pd.read_csv('cleansample_insdn.csv')

In [ ]:
print(data_clean['Label'].value_counts())

In [ ]:
labels = data_clean['Label'].values
data = data_clean.drop(columns=['Label'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler


X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

_features = X_train.shape[1]
# n_classes = labels.shape[1]

print('X.shape = ',X_train.shape)
print('Y.shape = ',labels.shape)
print('X_train.shape = ',X_train.shape)
# print('y_train.shape = ', Y_train.shape)
print('X_test.shape = ', X_test.shape)
# print('y_test.shape = ',Y_test.shape)

# print('X_val.shape = ', X_val.shape)
# print('y_val.shape = ',Y_val.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the VAE model
input_dim = X_train.shape[1]
latent_dim = 128 # Number of latent dimensions

# Encoder
inputs = layers.Input(shape=(input_dim,))
h = layers.Dense(128, activation='relu')(inputs)  # Additional layer
h = layers.Dense(64, activation='relu')(h) 
h = layers.Dense(32, activation='relu')(h)        # Additional layer
h = layers.Dense(16, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_var = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
decoder_h = layers.Dense(16, activation='relu')(z)
decoder_h = layers.Dense(32, activation='relu')(decoder_h)
decoder_h = layers.Dense(64, activation='relu')(decoder_h)  # Additional layer
decoder_h = layers.Dense(128, activation='relu')(decoder_h)  # Additional layer
decoder_mean = layers.Dense(input_dim, activation='relu')
h_decoded = decoder_h
x_decoded_mean = decoder_mean(h_decoded)

# VAE Model
vae = Model(inputs, x_decoded_mean)

# Loss Function
reconstruction_loss = tf.keras.losses.mean_squared_error(inputs, x_decoded_mean)
reconstruction_loss *= input_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

vae.compile(optimizer='adam')
vae.summary()


# Set up ModelCheckpoint to save the best model based on validation loss
checkpoint = ModelCheckpoint('best_VAE_model.h5', 
                             monitor='val_loss', 
                             save_best_only=True, 
                             mode='min', 
                             verbose=1)

In [ ]:
# # Train the VAE
# history = vae.fit(X_train[np.where(y_train==1)], X_train[np.where(y_train==1)], 
#                   shuffle=True, epochs=50, batch_size=32, validation_split=0.2)
import time

# Start the timer
start_time = time.time()

# Train the VAE
history = vae.fit(X_train[np.where(y_train==1)], X_train[np.where(y_train==1)], 
                  shuffle=True, epochs=50, batch_size=32, validation_split=0.2, callbacks=[checkpoint])


vae.load_weights('best_VAE_model.h5')
# End the timer
end_time = time.time()

# Calculate the training time
training_time = end_time - start_time

print(f"Test time: {training_time} seconds")

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()